In [6]:
search_for = 102
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.015831947326660156
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 332867491
type: [1, 1, 1, 1, 102] 102
cases of this type: 108243216
10000 79.68460727448145
20000 76.02315343953889
30000 75.97300534120538
40000 275.06724287198017
50000 255.46893391268958
60000 271.0794733120986
70000 267.13844689461564
80000 252.87001819580817
90000 218.31374282550834
100000 274.3527241392565
110000 285.6772062642048
120000 249.99795124161932
130000 225.37105430826642
140000 228.7350952920667
150000 223.4633806829422
160000 217.17607137784242
170000 236.37737718333764
180000 210.64884981833393
190000 220.7767932228826
200000 217.61719404231283
210000 222.8378173602947
220000 236.51215861013668
230000 215.23289796106863
240000 226.60948436226198
250000 214.7974216893686
260000 231.56183523533304
270000 259.1407411671261
280000 242.84819627186195
290000 243.76465389368403
300000 211.55070083823853
310000 217.1

3040000 203.4526282192431
3050000 218.0980832775137
3060000 220.17263646107702
3070000 213.80089840202123
3080000 229.06232515749628
3090000 220.33513284074292
3100000 235.53622569577624
3110000 208.60984856141084
3120000 196.99242747323595
3130000 219.60812428659284
3140000 230.08466882568695
3150000 100.57561999514901
3160000 73.98762213500282
3170000 89.62302137846632
3180000 73.29395932504639
3190000 187.25853714594737
3200000 147.2210215584495
3210000 64.70194418360934
3220000 70.68229699233133
3230000 58.374423446570496
3240000 58.24324871807094
3250000 135.24751502410217
3260000 230.30641468171385
3270000 209.57898677116003
3280000 186.5617659027801
3290000 214.92533747652098
3300000 200.14228305110439
3310000 204.3288718343768
3320000 234.44768641586717
3330000 203.90955123015888
3340000 223.60751070545172
3350000 217.0608255212347
3360000 205.99007990210185
3370000 217.20164719271978
3380000 198.3585520807583
3390000 198.18790979851732
3400000 223.69183687043403
3410000 202.94

6120000 215.1993639853143
6130000 217.90330542875412
6140000 235.4279145642251
6150000 207.36685595126377
6160000 202.85542748000046
6170000 212.82258823035676
6180000 198.6783269604906
6190000 210.11934086515768
6200000 190.16967751614794
6210000 185.57252475062734
6220000 218.24772809521696
6230000 197.3302819249194
6240000 193.57748373984325
6250000 217.35436370535447
6260000 200.85759683985177
6270000 216.55985668274198
6280000 204.49998551167093
6290000 220.79088206219882
6300000 167.49850827075534
6310000 72.34993491395619
6320000 71.76481907832786
6330000 70.79501515972379
6340000 71.01266369299294
6350000 145.85509836368183
6360000 258.4140263063002
6370000 236.57094747860958
6380000 224.49776441065617
6390000 196.12952220360904
6400000 231.62627029062534
6410000 220.11583132879372
6420000 58.384204689760054
6430000 57.622210806406734
6440000 57.652328495892206
6450000 57.50982845058214
6460000 71.86111906005752
6470000 242.24752178268977
6480000 216.18828796354234
6490000 216.

9210000 205.79507744446028
9220000 208.266865813676
9230000 184.72641339349022
9240000 207.32265474387236
9250000 195.29681891009258
9260000 203.7242219814146
9270000 221.87682848399592
9280000 187.13030076974343
9290000 192.35413371998285
9300000 209.97450338140712
9310000 185.37822701117383
9320000 201.05358042803581
9330000 203.84554943533445
9340000 195.10382771907635
9350000 211.94044391510576
9360000 190.35229608937516
9370000 193.15187839527172
9380000 211.62756179687634
9390000 190.56476421056436
9400000 212.84849391348837
9410000 205.03022227794648
9420000 196.5566208364468
9430000 209.7470404889162
9440000 204.18395374746436
9450000 171.1583036178652
9460000 80.07703313572368
9470000 78.31188322955838
9480000 69.94197565208846
9490000 69.95437818469439
9500000 78.8772573753045
9510000 266.108503093718
9520000 221.5574318059319
9530000 184.0772400384455
9540000 212.5126380816826
9550000 227.081915786242
9560000 230.1666782946929
9570000 209.09281067822056
9580000 205.169014554

12210000 28.106555325496284
12220000 28.334834756967187
12230000 28.207248314632423
12240000 157.4560158942972
12250000 227.0806077971334
12260000 188.519319361437
12270000 190.36083272844573
12280000 171.9552257029733
12290000 173.43360158039874
12300000 189.7446274960984
12310000 179.41232280000298
12320000 181.68679075572282
12330000 202.21181829323643
12340000 188.53861750946797
12350000 193.52840760084325
12360000 187.37985224404486
12370000 181.32976207056822
12380000 201.94181014249102
12390000 183.8058171942586
12400000 176.92762373652215
12410000 216.18451314368377
12420000 201.23176749420386
12430000 201.22298361817303
12440000 225.32961852730625
12450000 199.31076883276813
12460000 216.20670294303446
12470000 195.20424103363155
12480000 192.31376244124502
12490000 211.25659277746604
12500000 191.10657899854831
12510000 197.63160207167576
12520000 215.97131277357494
12530000 210.67284698550097
12540000 224.4896006012613
12550000 202.1074183361697
12560000 198.22326004499791
1

...

319690000 3.990569193891406
319700000 4.566983767967727
319710000 3.5020690582415264
319720000 3.0527027832350133
319730000 2.207625171814786
319740000 2.1596568179486924
319750000 3.783787920770645
319760000 3.515711646223128
319770000 3.994855679249313
319780000 3.9586086178215147
319790000 3.802237466447572
319800000 1.1095758321523401
319810000 3.653452054431458
319820000 3.3517600116412907
319830000 4.265340385370691
319840000 3.4458487012053136
319850000 3.9298448711355523
319860000 2.4626948541273475
319870000 2.782961793770651
319880000 2.4889529598540863
319890000 3.703037477329142
319900000 3.690472530338188
319910000 3.39721769524567
319920000 4.27416545562127
319930000 1.6264410178434847
319940000 2.9626418005312813
319950000 3.9262055825703936
319960000 4.362113568292638
319970000 3.6529417706023124
319980000 3.7094334428462052
319990000 2.0971872836516106
320000000 3.9103964718667927
320010000 1.8153505045114093
320020000 3.4213452571249006
320030000 3.6095231764160722
32

322570000 0.4652719072057804
322580000 0.4538522903927829
322590000 0.4330973545256191
322600000 0.5848437665433487
322610000 0.46245408716920344
322620000 0.7705545333137314
322630000 0.7853462646460533
322640000 0.7008078976588249
322650000 0.6853015417721802
322660000 0.6697246201171676
322670000 0.7104318631541927
322680000 0.5658456899678708
322690000 0.7579793286518056
322700000 0.756295693489737
322710000 0.7622827312874199
322720000 0.6201834687085748
322730000 0.7608567300297618
322740000 0.7741806569317249
322750000 0.4670185859587788
322760000 0.693373001786338
322770000 0.5091249870772957
322780000 0.46448305283208685
322790000 0.4263052241837713
322800000 0.44637429436989623
322810000 0.7394685493841172
322820000 0.4962407705643972
322830000 0.5012682785382337
322840000 0.7474171614340346
322850000 0.7622345198966397
322860000 0.6490246880982915
322870000 0.5539221207175851
322880000 0.4234175212675863
322890000 0.4210432059768968
322900000 0.5577008513530494
322910000 0.5

325430000 2.170691279509021
325440000 2.699527782412754
325450000 1.610432629161497
325460000 1.7828422375221518
325470000 2.397309537978742
325480000 2.041478401342988
325490000 1.860581640456471
325500000 1.1311258308466938
325510000 1.4426553432906868
325520000 2.1773402178966204
325530000 2.2574055646873115
325540000 2.250983407435993
325550000 1.3102630242830648
325560000 1.5497826257585354
325570000 2.7628067109265726
325580000 1.4574306412484843
325590000 2.0194562322663003
325600000 2.135885521215856
325610000 2.4272400346257954
325620000 2.3424940681041813
325630000 1.3347248814774353
325640000 0.9216506929756999
325650000 2.3254059583145645
325660000 2.189085735286812
325670000 1.8053300410779516
325680000 2.2864153973922927
325690000 0.9134728375380635
325700000 1.7085502030737532
325710000 1.5784093041279317
325720000 1.4250244403196572
325730000 2.0076406133798694
325740000 2.4010602777681815
325750000 2.3949321299731134
325760000 2.0951927694412666
325770000 0.62654532245

328250000 0.7259350824725429
328260000 0.8632327922295663
328270000 0.6146954771350026
328280000 0.34295517574572565
328290000 0.3006057446823584
328300000 0.31699294675296547
328310000 0.6193315191068849
328320000 0.6926439270078275
328330000 0.721784410737594
328340000 0.549619950441281
328350000 0.5727203286118574
328360000 0.7783211287724375
328370000 0.6288130078970988
328380000 0.594873790548
328390000 0.8534294646799564
328400000 0.6888349394837154
328410000 0.3086072764128711
328420000 0.3633824023874402
328430000 0.23367660824104813
328440000 0.5266639945115249
328450000 0.7361137181291779
328460000 0.7300105283296572
328470000 0.793734358020352
328480000 0.6385405875567794
best so far: 0
type: [2, 1, 1, 17, 3] 102
cases of this type: 397953
328490000 0.78182453120276
328500000 0.736125601565765
328510000 0.5823251030672789
328520000 0.7348581202022566
328530000 0.6896846790700357
328540000 0.6431085054957271
328550000 0.6344304540055196
328560000 0.5572877564088172
328570000 

330720000 0.16751135748016835
330730000 0.16780776431304217
330740000 0.1679899331422713
330750000 0.16337451459758148
330760000 0.16412490557505688
330770000 0.15816489620812735
330780000 0.1589255964729521
330790000 0.1433632086810668
330800000 0.13321845099982288
330810000 0.13783332107675075
330820000 0.15781558523654937
330830000 0.15366820979268023
330840000 0.15048188934648699
330850000 0.14143215073961019
330860000 0.1471438334861994
330870000 0.08873527825318442
330880000 0.12033561887055634
330890000 0.12732709054723715
330900000 0.2676587632592586
330910000 0.7990966984685659
330920000 0.420048768791808
330930000 0.32938057248959274
330940000 0.8575961396898825
330950000 0.6959404063830442
330960000 0.28785695289219754
330970000 0.41413674108848975
330980000 0.6006524354561501
330990000 0.487860699640625
331000000 0.26029833741325137
331010000 0.41375684880450697
331020000 0.4512857582093874
331030000 0.5174577975662351
331040000 0.3522585187147789
331050000 0.44862349644399